In [3]:
import pygame
from OpenGL.GL import *
from OpenGL.GLU import *
import math
import interface
import cv2
import numpy as np
import pygame.locals
import pygame.display
import pygame.image
class objItem(object):

    def __init__(self):
        self.angle = 0
        self.vertices = []
        self.faces = []
        self.coordinates = [-130, -450, -1300]  # [x,y,z]
        self.obj = interface.ObjLoader("obj_test.obj")
        self.position = [0, 0, -50]

    def render_scene(self):
        glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
        glShadeModel(GL_SMOOTH);
        glClearColor(0.0, 0.0, 0.0, 1.0)
        glMatrixMode(GL_MODELVIEW)
        glLoadIdentity()
        gluLookAt(0, 0, 0, math.sin(math.radians(self.angle)), 0, math.cos(math.radians(self.angle)) * -1, 0, 1, 0)
        glTranslatef(self.coordinates[0], self.coordinates[1], self.coordinates[2])

    def move_forward(self):
        self.coordinates[2] += 10 * math.cos(math.radians(self.angle))
        self.coordinates[0] -= 10 * math.sin(math.radians(self.angle))

    def move_back(self):  
        self.coordinates[2] -= 10 * math.cos(math.radians(self.angle))
        self.coordinates[0] += 10 * math.sin(math.radians(self.angle))

    def move_left(self, n):
        self.coordinates[0] += n * math.cos(math.radians(self.angle))
        self.coordinates[2] += n * math.sin(math.radians(self.angle))

    def move_right(self, n):
        self.coordinates[0] -= n * math.cos(math.radians(self.angle))
        self.coordinates[2] -= n * math.sin(math.radians(self.angle))
        print(self.coordinates[0])
        print(self.coordinates[2])
    def rotate(self, n):
        self.angle += n 

    def fullRotate(self):  
        for i in range(0, 36):
            self.angle += 10
            self.move_left(10)
            self.render_scene()
            self.obj.render_scene()
            pygame.display.flip()
        ##                

def main():
    width = 800
    height = 600
    pygame.init()
    
    pygame.display.gl_set_attribute(pygame.locals.GL_MULTISAMPLEBUFFERS,1)
    pygame.display.gl_set_attribute(pygame.locals.GL_MULTISAMPLESAMPLES,16)
    window = pygame.display.set_mode((width, height), pygame.DOUBLEBUF | pygame.OPENGL)
    pygame.display.set_caption("Python furniture module")
    clock = pygame.time.Clock()
    # Function checker
    glDisable(GL_TEXTURE_2D)
    glEnable(GL_DEPTH_TEST)
    glEnable(GL_CULL_FACE)

    #
    glMatrixMode(GL_PROJECTION)
    gluPerspective(45.0, float(width) / height, .1, 10000.)
    glMatrixMode(GL_MODELVIEW)
    #
    meshObj = objItem()
    # - Event Loop - #
    done = False
    while not done:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                done = True
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_LEFT or event.key == pygame.K_a:
                    meshObj.move_left(10)
                elif event.key == pygame.K_RIGHT or event.key == pygame.K_d:
                    meshObj.move_right(10)
                elif event.key == pygame.K_UP or event.key == pygame.K_w:
                    meshObj.move_forward()
                elif event.key == pygame.K_DOWN or event.key == pygame.K_s:
                    meshObj.move_back()
                elif event.key == pygame.K_1:
                    meshObj.rotate(10)
                    meshObj.move_left(10)
                elif event.key == pygame.K_2:
                    meshObj.rotate(-10)
                    meshObj.move_right(10)
                    #print(meshObj.coordinates[0],' ', print(meshObj.coordinates[1]),' ',print(meshObj.coordinates[2]))
                elif event.key == pygame.K_3:
                    meshObj.fullRotate()
        glEnable(GL_BLEND)        
        glBlendFunc(GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA)
        #glEnable(GL_MULTISAMPLE)
        #glDisable(GL_DEPTH_TEST);
        
        #glHint(GL_POLYGON_SMOOTH_HINT, GL_NICEST);
        #glEnable(GL_POLYGON_SMOOTH)

        
        
        meshObj.render_scene()
        meshObj.obj.render_scene()
        pygame.display.flip()
        clock.tick(30)
    pygame.image.save(window, "normal.png")
    pygame.quit()


if __name__ == '__main__':
    main()

In [1]:
import cv2
import numpy as np

src = cv2.imread("normal.png", cv2.IMREAD_GRAYSCALE)
#gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)


laplacian = cv2.Laplacian(src, cv2.CV_8U, ksize = 5)

dst = cv2.bitwise_not(laplacian)
#dst = cv2.bilateralFilter(dst,15,75,75)
#dst = cv2.blur(dst,(3,3))
dst = cv2.GaussianBlur(dst, (5,5), 0)
#dst = cv2.medianBlur(dst, 3)
cv2.imshow("dst", dst)
cv2.imwrite('instruction.png', dst)
cv2.waitKey(0)
cv2.destroyAllWindows()